# Construct Korean DPR Dataset (KorQuad version 1)

In [1]:
import json
import pandas as pd
from tqdm import tqdm
from nltk import sent_tokenize
from rank_bm25 import BM25Okapi
from datasets import load_dataset
from transformers import AutoTokenizer

### 1. Load Dataset

In [2]:
dataset = load_dataset('KorQuAD/squad_kor_v1', trust_remote_code=True)

In [3]:
dataset['train'][1]

{'id': '6566495-0-1',
 'title': '파우스트_서곡',
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'question': '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?',
 'answers': {'text': ['1악장'], 'answer_start': [421]}}

### 2. Chunk Contexts

In [4]:
def chunk_context(context, num_sents, overlaps):
    chunks = []
    start, end = 0, num_sents
    
    total_sents = sent_tokenize(context)

    while start < len(total_sents):
        chunk = total_sents[start:end]
        chunks.append(' '.join(chunk))
        
        start += (num_sents - overlaps)
        end = start + num_sents

    return chunks

In [5]:
sample_ctx = dataset['train'][0]['context']

chunks = chunk_context(sample_ctx, 3, 1)
chunks

['1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다.',
 '또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다.',
 '그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다.',
 '결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.']

### 3. Fill Templates

In [6]:
TEMPLATE_ALL = {
    "question": "",
	"answers": [],
	"positive": [],
  }

TEMPLATE_TEXT = {
    "title": "",
    "text": "",
}

In [7]:
title = dataset['train'][0]['title']
title

'파우스트_서곡'

In [8]:
question = dataset['train'][0]['question']
question

'바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'

In [9]:
answer = dataset['train'][0]['answers']['text'][0]
answer

'교향곡'

In [10]:
def fill_templates(chunks, title, question, answer, idx):
    positives = []
    for chunk in chunks:
        template_instance1 = TEMPLATE_TEXT.copy()
        template_instance1['title'] = title
        template_instance1['text'] = chunk
    
        if answer in chunk:
            positives.append(template_instance1.copy())

    template_instance2 = TEMPLATE_ALL.copy()
    template_instance2['question'] = question
    template_instance2['answers'] = [answer]
    template_instance2['positive'] = positives
    
    return template_instance2

In [11]:
fill_templates(chunks, title, question, answer, 1)

{'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': ['교향곡'],
 'positive': [{'title': '파우스트_서곡',
   'text': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다.'},
  {'title': '파우스트_서곡',
   'text': '또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다.'},
  {'title': '파우스트_서곡',
   'text': '그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다.'}]}

### 4. Construct DPR Dataset

In [12]:
TEMPLATE_ALL = {
    "question": "",
	"answers": [],
	"positive": [],
  }

TEMPLATE_TEXT = {
    "title": "",
    "text": "",
}

In [13]:
def construct_dpr_dataset(dataset, num_sents, overlaps):
    dpr_dataset = []
    for idx in tqdm(range(len(dataset))):
        sample = dataset[idx]
        
        title = sample['title']
        question = sample['question']
        answer = sample['answers']['text'][0]
        chunks = chunk_context(sample['context'], num_sents, overlaps)
        
        output = fill_templates(chunks, title, question, answer, idx+1)
        dpr_dataset.append(output)
    
    return dpr_dataset

In [14]:
dpr_dataset = construct_dpr_dataset(dataset['train'], 3, 1)

100%|███████████████████████████████████████████████████████████████████████████| 60407/60407 [00:13<00:00, 4571.48it/s]


In [15]:
dpr_dataset[0]

{'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': ['교향곡'],
 'positive': [{'title': '파우스트_서곡',
   'text': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다.'},
  {'title': '파우스트_서곡',
   'text': '또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다.'},
  {'title': '파우스트_서곡',
   'text': '그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다.'}]}

### 5. Set Index

In [16]:
def set_index_to_text(dataset):
    text_2_index = {}
    text_2_title = {}
    current_idx = 0  # Initialize the starting index

    for sample in dataset:
        pos_text = [pos['text'] for pos in sample['positive']]
        pos_title = [pos['title'] for pos in sample['positive']]
    
        all_idx = []
        for text, title in zip(pos_text, pos_title):
            if text not in text_2_index:
                text_2_index[text] = current_idx 
                text_2_title[text] = title  # Map the text to its title
                current_idx += 1

        for pos in sample['positive']:
            pos['idx'] = text_2_index[pos['text']]
            all_idx.append(text_2_index[pos['text']])
        
        sample['answer_idx'] = all_idx

    return dataset, text_2_index, text_2_title

In [17]:
dpr_dataset, text_2_index, text_2_title = set_index_to_text(dpr_dataset)

In [18]:
dpr_dataset[0]

{'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': ['교향곡'],
 'positive': [{'title': '파우스트_서곡',
   'text': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다.',
   'idx': 0},
  {'title': '파우스트_서곡',
   'text': '또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다.',
   'idx': 1},
  {'title': '파우스트_서곡',
   'text': '그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다.',
   'idx': 2}],
 'an

### 5. Add Hard Negative (Optional)

In [38]:
def get_hard_negative(dataset, text_2_index, text_2_title):
    corpus = list(text_2_index.keys())
    tokenized_corpus = [context.split(" ") for context in corpus]
    bm25 = BM25Okapi(tokenized_corpus)

    for sample in tqdm(dataset, desc="Processing samples"):
        question = sample['question']
        positive = sample['positive']
        
        top_n = bm25.get_top_n(question.split(" "), corpus, n=30)  # Increase n to have more candidates
        
        hard_neg = []
        positive_idx_set = set(sample['answer_idx'])
        for doc in top_n:
            if text_2_index[doc] not in positive_idx_set:
                hard_neg.append({'title': text_2_title[doc],
                                 'text': doc,
                                 'idx': text_2_index[doc]})
            
            if len(hard_neg) >= len(positive):
                break
        
        # If not enough hard negatives, repeat existing ones to match the number of positives
        if len(hard_neg) < len(positive):
            print(f"Warning: Not enough hard negatives for question: {question}. Duplicating existing hard negatives.")
            while len(hard_neg) < len(positive):
                hard_neg.extend(hard_neg[:len(positive) - len(hard_neg)])
        
        sample['hard_neg'] = hard_neg[:len(positive)]  # Trim to ensure exact number matches positive samples
        
    return dataset

In [39]:
dpr_dataset = get_hard_negative(dpr_dataset, text_2_index, text_2_title)

Processing samples: 100%|█████████████████████████████████████████████████████████| 60407/60407 [37:36<00:00, 26.77it/s]


In [40]:
dpr_dataset[0]

{'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': ['교향곡'],
 'positive': [{'title': '파우스트_서곡',
   'text': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다.',
   'idx': 0},
  {'title': '파우스트_서곡',
   'text': '또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다.',
   'idx': 1},
  {'title': '파우스트_서곡',
   'text': '그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다.',
   'idx': 2}],
 'an

### 6. Save to Json

In [41]:
file_path = 'korquad_v1_train.json'

In [42]:
with open(file_path, 'w') as outfile:
    json.dump(dpr_dataset, outfile, indent=4)

In [43]:
with open(file_path) as infile:
    dataset = json.load(infile)

In [44]:
dataset[0]

{'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': ['교향곡'],
 'positive': [{'title': '파우스트_서곡',
   'text': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다.',
   'idx': 0},
  {'title': '파우스트_서곡',
   'text': '또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다.',
   'idx': 1},
  {'title': '파우스트_서곡',
   'text': '그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다.',
   'idx': 2}],
 'an

In [45]:
index_path = 'korquad_v1_train_index.json'

In [46]:
with open(index_path, 'w') as outfile:
    json.dump(text_2_index, outfile, indent=4)

In [47]:
with open(index_path) as infile:
    index = json.load(infile)

In [48]:
len(index)

30738